# Demonstrateur pour la plateforme web-cartographique d'indicateurs  - 
# #3. illustration des évolutions temporelles d'un indicateur

Ce notebook a pour objectif de proposer un premier aperçu visuel de certains indicateurs à travers des cartes interactives.   
Ces visualisations serviront de base de discussion pour la conception de la plateforme web dédiée.  

Les données sont extraites depuis notre base de données **PostgreSQL/PostGIS**.

🛠️ Les principaux outils utilisés sont :
- **Pandas** : pour le traitement et la manipulation des données à partir de la BDD.
- **Folium** : pour la création de cartes interactives en Python.

📋 Deux types de visualisations sont explorés :
1. **DualMap** : permet de comparer côte à côte deux années différentes pour observer les évolutions spatiales d’un indicateur.
2. **TimeSliderMap** : permet de faire défiler les années facilement pour suivre la progression d’un indicateur dans le temps.

Indicateurs illustrés dans ce notebook :
- **Surface en agriculture biologique** : indicateur simple.  
- **HVN** : indicateur plus complexe , ajout de liens vers une documentation précise.

Ce notebook est un prototype visant à tester la lisibilité et la pertinence des représentations cartographiques.  

In [ ]:
import psycopg2
import geopandas as gpd
import pandas as pd
import numpy as np
import json

# pour la visualisation :
from matplotlib import colormaps
import matplotlib.colors as mcolors
import branca.colormap as cm
from branca.colormap import linear
from branca.element import Element
import folium
from folium.plugins import FloatImage
from folium.plugins import DualMap
from folium.plugins import TimeSliderChoropleth
from folium.plugins.float_image import FloatImage
from branca.element import MacroElement, Template
from folium.plugins import Search


## Connexion à la BDD

In [ ]:
def connection_2_bdd(uname, password, host, port, database_name):
    """Connection via psycopg2 à la BDD PostgreSQL"""
    return psycopg2.connect(
        user=uname, password=password, host=host, port=port, database=database_name
    )

In [ ]:
# Lecture des params de connexion dans un json dédié
fileObject = open("../param_connexion_BDD.json", mode="r", encoding="utf-8")
jsonContent = fileObject.read()
param_connexion = json.loads(jsonContent)

# Connexion a la BDD
conn = connection_2_bdd(
    param_connexion["username"],
    param_connexion["password"],
    param_connexion["host"],
    param_connexion["port"],
    param_connexion["bdd_name"],
)
curs = conn.cursor()

## Récupération de la zone d'intérêt

In [ ]:
# Requête SQL pour récupérer toutes les données
sql = 'SELECT * FROM app_webae_dev.territoire_test_bv_cerou_vere UNION SELECT * FROM app_webae_dev.territoire_test_bv_viaur;'

gdf_aac = gpd.read_postgis(sql, conn, geom_col="geom")

# Conversion de la projection
gdf_aac = gdf_aac.to_crs(epsg=4326)
gdf_aac = gdf_aac[['geom']]

## Récupération des indicateurs par communes

In [ ]:
dict_gdf_communes = dict()

# Boucle pour récupérer les données de 2015 à 2023
for year in range(2015, 2024):
    # Requête SQL pour récupérer toutes les communes et indicateurs sur le territoire
    sql = f"""
    WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
    UNION
    SELECT *
    FROM app_webae_dev.territoire_test_bv_viaur
    ),
    communes_territoires AS
    (SELECT DISTINCT 'aac_coulonge_st_hyppolyte' AS territoire
    ,b.insee_com,nom_commune,insee_dep,nom_departement,insee_reg,nom_region,siren_epci,nom_epci,b.geom
          FROM territoire_limite a
          JOIN app_webae_dev.v_communes2025_fre_4326 b
          ON st_intersects(st_transform(a.geom,4326),st_transform(b.geom,4326))
    )
    SELECT DISTINCT a.geom,a.nom_commune, b.*
    FROM communes_territoires a
    LEFT JOIN app_webae_dev.mv_indicateurs{year}_com2025_fr_rpg_seul b
    ON a.insee_com=b.insee_com;
    """   
    # Lecture dans un GeoDataFrame via geopandas
    gdf_communes = gpd.read_postgis(sql, conn, geom_col="geom")
    
    # Conversion de la projection
    gdf_communes = gdf_communes.to_crs(epsg=4326)
    
    # Ajout du GeoDataFrame dans le dictionnaire
    dict_gdf_communes[f'gdf_communes_{year}'] = gdf_communes

In [ ]:
gdf_communes_2020 = dict_gdf_communes['gdf_communes_2020']
gdf_communes_2023 = dict_gdf_communes['gdf_communes_2023']

## Récupération des indicateurs par Carreaux de 10km

In [ ]:
dict_gdf_carreaux = dict()

for year in range(2015, 2024):
    # Requête SQL pour récupérer toutes les communes et indicateurs sur le territoire
    sql = f"""
    WITH territoire_limite AS (
        SELECT * 
        FROM app_webae_dev.territoire_test_bv_cerou_vere
        UNION
        SELECT *
        FROM app_webae_dev.territoire_test_bv_viaur
    ),
    carreaux_territoire AS (
        SELECT DISTINCT c.id_carreau, c.geom
        FROM app_webae_dev.ign_carreaux_10km_fr_2154 c
        JOIN territoire_limite t
        ON ST_Intersects(c.geom, t.geom)
    )
    SELECT c.geom, i.*
    FROM carreaux_territoire c
    JOIN app_webae_dev.mv_indicateurs{year}_carreau10km_fr_rpg_seul i
    ON c.id_carreau = i.id_carreau;
    """

    # Lecture dans un GeoDataFrame via geopandas
    gdf_carreaux = gpd.read_postgis(sql, conn, geom_col="geom")

    # Conversion de la projection
    gdf_carreaux = gdf_carreaux.to_crs(epsg=4326)

    # Ajout du GeoDataFrame dans le dictionnaire
    dict_gdf_carreaux[f'gdf_carreaux_{year}'] = gdf_carreaux

In [ ]:
gdf_carreaux_2020 = dict_gdf_carreaux['gdf_carreaux_2020']
gdf_carreaux_2023 = dict_gdf_carreaux['gdf_carreaux_2023']

In [ ]:
gdf_communes = gpd.GeoDataFrame()
gdf_carreaux = gpd.GeoDataFrame()

for year in range(2015, 2024):
    # Ajout d'une colonne pour calcluer le pourcentage de Surface en Herbe en 2015
    gdf_communes[f"pct_sth_pt_{year}"] = (100 * dict_gdf_communes[f'gdf_communes_{year}']["part_sth_pt"]).round(2)
    gdf_carreaux[f"pct_sth_pt_{year}"] = (100 * dict_gdf_carreaux[f'gdf_carreaux_{year}']["part_sth_pt"]).round(2)
    # Ajoutd'une colonne pour calculer la moyenne de la Surface en Herbe en 2015 par unité de surface
    gdf_communes[f"pct_sth_pt{year}_moyen"] = gdf_communes[f"pct_sth_pt_{year}"].mean().round(2)
    gdf_carreaux[f"pct_sth_pt{year}_moyen"] = gdf_carreaux[f"pct_sth_pt_{year}"].mean().round(2)

gdf_communes["geom"] = dict_gdf_communes[f'gdf_communes_2023']["geom"]
gdf_carreaux["geom"] = dict_gdf_carreaux[f'gdf_carreaux_2023']["geom"]
gdf_communes["insee_com"] = dict_gdf_communes[f'gdf_communes_2023']["insee_com"]
gdf_carreaux["id_carreau"] = dict_gdf_carreaux[f'gdf_carreaux_2023']["id_carreau"]
gdf_communes["nom_commune"] = dict_gdf_communes[f'gdf_communes_2023']["nom_commune"]

## DualMap pour comparaison

Comparaison du pourcentage de SAUbio entre 2020 et 2023.

In [ ]:
# Ajout d'une colonne pour calcluer le pourcentage de SAU bio en 2020
gdf_communes_2020["pourcentage_bio_2020"] = (100 * gdf_communes_2020["sau_rpgbio"] / gdf_communes_2020["sau_totale"]).round(2)
gdf_carreaux_2020["pourcentage_bio_2020"] = (100 * gdf_carreaux_2020["sau_rpgbio"] / gdf_carreaux_2020["sau_totale"]).round(2)

# Ajoutd'une colonne pour calculer la moyenne de la SAU bio en 2020 par unité de surface
gdf_communes_2020["pourcentage_bio_2020_moyen"] = gdf_communes_2020["pourcentage_bio_2020"].mean().round(2)
gdf_carreaux_2020["pourcentage_bio_2020_moyen"] = gdf_carreaux_2020["pourcentage_bio_2020"].mean().round(2)

# Ajout d'une colonne pour calcluer le pourcentage de SAU bio en 2023
gdf_communes_2020["pourcentage_bio_2023"] = (100 * gdf_communes_2023["sau_rpgbio"] / gdf_communes_2023["sau_totale"]).round(2)
gdf_carreaux_2020["pourcentage_bio_2023"] = (100 * gdf_carreaux_2023["sau_rpgbio"] / gdf_carreaux_2023["sau_totale"]).round(2)

# Ajoutd'une colonne pour calculer la moyenne de la SAU bio en 2023 par unité de surface
gdf_communes_2020["pourcentage_bio_2023_moyen"] = gdf_communes_2020["pourcentage_bio_2023"].mean().round(2)
gdf_carreaux_2020["pourcentage_bio_2023_moyen"] = gdf_carreaux_2020["pourcentage_bio_2023"].mean().round(2)

# Ajout d'une colonne pour calculer le différence entre 2023 et 2020
gdf_communes_2020["diff_2023_2020"] = (gdf_communes_2020["pourcentage_bio_2023"] - gdf_communes_2020["pourcentage_bio_2020"]).round(2)
gdf_carreaux_2020["diff_2023_2020"] = (gdf_carreaux_2020["pourcentage_bio_2023"] - gdf_carreaux_2020["pourcentage_bio_2020"]).round(2)

In [ ]:
def get_colormap(vmin, vmax, cmap_name='Greens', caption=''):
    """Crée une colormap"""
    mpl_colormap = colormaps.get_cmap(cmap_name)
    
    return cm.LinearColormap(
        colors=[mpl_colormap(i) for i in range(mpl_colormap.N)],
        vmin=vmin, vmax=vmax,
        caption=caption,
    ), vmin, vmax

def style_function_factory(colormap, col):
    """Crée une fonction de style pour la coloration selon une colonne."""
    def style_function(feature):
        value = feature['properties'].get(col)
        color = colormap(value) if value is not None else 'grey'
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        }
    return style_function

def add_geojson_layer(map_obj, gdf, name, colormap, col, tooltip_fields, tooltip_aliases):
    style_fn = style_function_factory(colormap, col)

    def highlight_function(feature):
        value = feature['properties'].get(col)
        color = colormap(value) if value is not None else 'grey'
        return {
            'fillColor': color,
            'color': color,
            'weight': 3,
            'fillOpacity': 0.7,
        }

    folium.GeoJson(
        data=gdf.to_json(),
        name=name,
        show=False,
        style_function=style_fn,
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
    ).add_to(map_obj)

# Centrage de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]
m = DualMap(location=map_center, tiles=None, zoom_start=8)

for side in [m.m1, m.m2]:
    folium.TileLayer("OpenStreetMap").add_to(side)
    # Ajout de la couche AAC
    folium.GeoJson(
        data=gdf_aac.to_json(),
        name="CEROU-VERE et VIAUR",
        style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.2},
        highlight_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 3, 'fillOpacity': 0.7},
    ).add_to(side)

# ---------- BIO 2020 ----------
col_bio_2020 = 'pourcentage_bio_2020'
col_bio_2023 = 'pourcentage_bio_2023'
vmin = gdf_communes_2020[col_bio_2020].quantile(0.05).round(1)
vmax = gdf_communes_2020[col_bio_2020].quantile(0.95).round(1)
colormap_bio, _, _ = get_colormap(vmin, vmax, caption='Pourcentage SAU bio 2020 (%)')

# Créer un template HTML pour afficher la colormap_bio à droite
legend_html = colormap_bio._repr_html_()
legend_template = Template(f"""
{{% macro html(this, kwargs) %}}
<div style="position: fixed;
            top: 70px; left: 0vw; width: 200px; z-index:9999;
            background-color: transparent; padding: 10px; border:0px ;">
    {legend_html}
</div>
{{% endmacro %}}
""")
legend_element = MacroElement()
legend_element._template = legend_template
# Ajouter ce composant à la carte de droite
m.m1.get_root().add_child(legend_element)

add_geojson_layer(m.m1, gdf_carreaux_2020, f"{col_bio_2020}_carreaux", colormap_bio, col_bio_2020,
                  ['id_carreau', col_bio_2020, 'sau_totale'],
                  ['Carreau', 'Pourcentage SAU bio 2020 (%)','SAU totale (ha)'])
add_geojson_layer(m.m1, gdf_communes_2020, f"{col_bio_2020}_communes", colormap_bio, col_bio_2020,
                  ['nom_commune', col_bio_2020, 'sau_totale'],
                  ['Commune', 'Pourcentage SAU bio 2020 (%)', 'SAU totale (ha)'])

# ---------- Différence BIO 2020 - BIO 2023 ----------
col_diff_bio = 'diff_2023_2020'
vmin = -10
vmax = 10
colormap_diff_bio, _, _ = get_colormap(vmin, vmax, cmap_name='RdBu', caption='Différence pourcentage SAU bio 2023/2020 (%)')
from branca.element import MacroElement, Template
# Créer un template HTML pour afficher la colormap_herbe à droite
legend_html = colormap_diff_bio._repr_html_()
legend_template = Template(f"""
{{% macro html(this, kwargs) %}}
<div style="position: fixed;
            top: 70px; right: 35vw; width: 200px; z-index:9999;
            background-color: transparent; padding: 10px; border:0px ;">
    {legend_html}
</div>
{{% endmacro %}}
""")
legend_element = MacroElement()
legend_element._template = legend_template
# Ajouter ce composant à la carte de droite
m.m2.get_root().add_child(legend_element)
# colormap_diff_bio.add_to(m.m2)

add_geojson_layer(m.m2, gdf_carreaux_2020, f"{col_diff_bio}_carreaux", colormap_diff_bio, col_diff_bio,
                  ['id_carreau', col_diff_bio, col_bio_2020, col_bio_2023, 'sau_totale'],
                  ['Carreau', 'Différence pourcentage SAU bio (%)', 'Pourcentage SAU bio 2020 (%)', 'Pourcentage SAU bio 2023 (%)', 'SAU totale (ha)'])
add_geojson_layer(m.m2, gdf_communes_2020, f"{col_diff_bio}_communes", colormap_diff_bio, col_diff_bio,
                  ['nom_commune', col_diff_bio, col_bio_2020, col_bio_2023, 'sau_totale'],
                  ['Commune', 'Différence pourcentage SAU bio (%)', 'Pourcentage SAU bio 2020 (%)', 'Pourcentage SAU bio 2023 (%)', 'SAU totale (ha)'])

# Contrôles
folium.LayerControl(collapsed=False).add_to(m)

m.save("carte_DualMap_EvolutionsSAUBio.html")

## Time Slider Chrolopleth HVN (diversité d'assolement) par communes entre 2015 - 2023

In [ ]:
gdf_communes = gpd.GeoDataFrame()
gdf_carreaux = gpd.GeoDataFrame()

for year in range(2015, 2024):
    # Ajout d'une colonne pour calcluer le pourcentage de Surface en Herbe
    gdf_communes[f"hvn_i1_grpa_{year}"] = (dict_gdf_communes[f'gdf_communes_{year}']["hvn_i1_grpa"]).round(2)
    gdf_carreaux[f"hvn_i1_grpa_{year}"] = (dict_gdf_carreaux[f'gdf_carreaux_{year}']["hvn_i1_grpa"]).round(2)

gdf_communes["geom"] = dict_gdf_communes[f'gdf_communes_2023']["geom"]
gdf_carreaux["geom"] = dict_gdf_carreaux[f'gdf_carreaux_2023']["geom"]
gdf_communes["insee_com"] = dict_gdf_communes[f'gdf_communes_2023']["insee_com"]
gdf_carreaux["id_carreau"] = dict_gdf_carreaux[f'gdf_carreaux_2023']["id_carreau"]
gdf_communes["nom_commune"] = dict_gdf_communes[f'gdf_communes_2023']["nom_commune"]

In [ ]:
# Chargement et préparation des données
gdf = gdf_communes.copy()
gdf['insee_com'] = gdf['insee_com'].astype(str)
gdf = gdf.set_geometry("geom")

# Préparation timestamps
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
value_columns = [f'hvn_i1_grpa_{y}' for y in years]
timestamps = pd.to_datetime([f'{y}-01-01' for y in years])
timestamps = timestamps.astype("int64") // 10**9
dt_index = timestamps.astype("U10")

from branca.colormap import StepColormap

# Définir les bornes unitaires de 0 à 10
boundaries = list(range(0, 11))

# Définir 10 couleurs du rouge (0) au vert (10)
colors = [
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
    '#006837',
    '#004529'
]

# Créer la colormap à paliers ci-dessus
colormap = StepColormap(
    colors=colors,
    index=boundaries,
    vmin=0,
    vmax=10,
    caption="Score de diversité d'assolement (0-10)",
)

# Création du styledict pour le TimeSlider
styledict = {}
for idx, row in gdf.iterrows():
    feature_id = str(row['insee_com'])
    styledict[feature_id] = {}
    for timestamp, col in zip(dt_index, value_columns):
        val = row.get(col, None)
        color = colormap(val) if pd.notnull(val) else "#d3d3d3"
        styledict[feature_id][str(timestamp)] = {
            "color": color,
            "opacity": 0.7,
        }

# Préparation du GeoDataFrame pour le TimeSlider
new_gdf = gdf[['geom']].reset_index(drop=True).drop_duplicates()
new_gdf.index = list(styledict.keys())
new_gdf = gdf.set_geometry('geom')

# Création de la carte centrée
center = new_gdf.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=8, tiles="OpenStreetMap")

# Ajout du TimeSliderChoropleth
geojson_data = json.loads(new_gdf.to_json())
for i, feature in enumerate(geojson_data["features"]):
    feature["id"] = str(gdf.iloc[i]["insee_com"])

TimeSliderChoropleth(
    data=geojson_data,
    styledict=styledict,
).add_to(m)


# Ajout des popups
folium.GeoJson(
    gdf,
    name="Popups",
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    },
    popup=folium.GeoJsonPopup(
        fields=["nom_commune", "hvn_i1_grpa_2015", "hvn_i1_grpa_2016", "hvn_i1_grpa_2017", "hvn_i1_grpa_2018", "hvn_i1_grpa_2019", "hvn_i1_grpa_2020", "hvn_i1_grpa_2021", "hvn_i1_grpa_2022", "hvn_i1_grpa_2023"],
        aliases=["Commune :", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement"],
        localize=True,
        labels=True,
    )
).add_to(m)

# Ajout de la légende
colormap.add_to(m)
# Ajout des boutons pour afficher les précautions et la description
html = """
<style>
#info-box {
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;  /* caché par défaut */
}
#toggle-btn {
    position: fixed;
    bottom: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="info-box">
    <b>Précautions de lecture :</b><br>
    ...
</div>

<button id="toggle-btn" onclick="
    var box = document.getElementById('info-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer les précautions';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher les précautions';
    }
">Afficher les précautions</button>
"""
m.get_root().html.add_child(Element(html))

html2 = """
<style>
#desc-box {
    position: fixed;
    top: 100px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;
}
#desc-btn {
    position: fixed;
    top: 70px;
    left: 50px;
    z-index: 10000;
    background-color: #2196F3;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="desc-box">
    <b>Description de l’indicateur :</b><br>
    Cet indicateur (noté sur 10) évalue la diversité de l’assolement et la part des prairies dans l’assolement.
    <br>
    <a href="https://solagro.org/images/imagesCK/files/documents/2021_Guide_me__thodologique_HVN.pdf"
        target="_blank" style="color: #1a73e8; text-decoration: underline;">
        Voir le guide méthodologique (PDF)
    </a>
</div>

<button id="desc-btn" onclick="
    var box = document.getElementById('desc-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer la description';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher la description';
    }
">Afficher la description</button>
"""
m.get_root().html.add_child(Element(html2))

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=2, right=2, width='150px').add_to(m)

# GeoJson utilisé pour la recherche
geojson_search = folium.GeoJson(
    gdf,
    name="Recherche",
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["nom_commune", "hvn_i1_grpa_2015", "hvn_i1_grpa_2016", "hvn_i1_grpa_2017", "hvn_i1_grpa_2018", "hvn_i1_grpa_2019", "hvn_i1_grpa_2020", "hvn_i1_grpa_2021", "hvn_i1_grpa_2022", "hvn_i1_grpa_2023"],
        aliases=["Commune :", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement"],
        )
).add_to(m)

# Plugin Search
Search(
    layer=geojson_search,
    search_label="nom_commune",
    placeholder="Rechercher une commune",
    collapsed=False
).add_to(m)

m.save("carte_Timeslider_HVN_commune.html")

## Time Slider Chrolopleth Diversité Assolement (HVN) par Carreaux de 10km entre 2015 - 2023

In [ ]:
# Chargement et préparation des données
gdf = gdf_carreaux.copy()
gdf['id_carreau'] = gdf['id_carreau'].astype(str)
gdf = gdf.set_geometry("geom")

# Préparation des timestamps
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
value_columns = [f'hvn_i1_grpa_{y}' for y in years]
timestamps = pd.to_datetime([f'{y}-01-01' for y in years])
timestamps = timestamps.astype("int64") // 10**9
dt_index = timestamps.astype("U10")

# Création de la colormap à partir de 2023
from branca.colormap import StepColormap

boundaries = list(range(0, 11))

colors = [
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
    '#006837',
    '#004529'
]

# Créer la colormap à paliers unitaires
colormap = StepColormap(
    colors=colors,
    index=boundaries,
    vmin=0,
    vmax=10,
    caption="Score de diversité d'assolement (0-10)",
)

# Création du styledict pour le TimeSlider
styledict = {}
for idx, row in gdf.iterrows():
    feature_id = str(row['id_carreau'])
    styledict[feature_id] = {}
    for timestamp, col in zip(dt_index, value_columns):
        val = row.get(col, None)
        color = colormap(val) if pd.notnull(val) else "#d3d3d3"
        styledict[feature_id][str(timestamp)] = {
            "color": color,
            "opacity": 0.7,
        }

# Préparation du GeoDataFrame pour le TimeSlider
new_gdf = gdf[['geom']].reset_index(drop=True).drop_duplicates()
new_gdf.index = list(styledict.keys())
new_gdf = gdf.set_geometry('geom')

# Création de la carte centrée
center = new_gdf.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=8, tiles="OpenStreetMap")

# Ajout du TimeSliderChoropleth
geojson_data = json.loads(new_gdf.to_json())
for i, feature in enumerate(geojson_data["features"]):
    feature["id"] = str(gdf.iloc[i]["id_carreau"])

TimeSliderChoropleth(
    data=geojson_data,
    styledict=styledict,
).add_to(m)


# Ajout des popups
folium.GeoJson(
    gdf,
    name="Popups",
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    },
    popup=folium.GeoJsonPopup(
        fields=["id_carreau", "hvn_i1_grpa_2015", "hvn_i1_grpa_2016", "hvn_i1_grpa_2017", "hvn_i1_grpa_2018", "hvn_i1_grpa_2019", "hvn_i1_grpa_2020", "hvn_i1_grpa_2021", "hvn_i1_grpa_2022", "hvn_i1_grpa_2023"],
        aliases=["id_carreau :", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement", "Score Divesité Assolement"],
        localize=True,
        labels=True,
    )
).add_to(m)

# Ajout de la légende
colormap.add_to(m)
# Ajout des boutons pour afficher les précautions et la description
html = """
<style>
#info-box {
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;  /* caché par défaut */
}
#toggle-btn {
    position: fixed;
    bottom: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="info-box">
    <b>Précautions de lecture :</b><br>
    ...
</div>

<button id="toggle-btn" onclick="
    var box = document.getElementById('info-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer les précautions';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher les précautions';
    }
">Afficher les précautions</button>
"""
m.get_root().html.add_child(Element(html))

html2 = """
<style>
#desc-box {
    position: fixed;
    top: 100px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;
}
#desc-btn {
    position: fixed;
    top: 70px;
    left: 50px;
    z-index: 10000;
    background-color: #2196F3;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="desc-box">
    <b>Description de l’indicateur :</b><br>
    Cet indicateur (noté sur 10) évalue la diversité de l’assolement et la part des prairies dans l’assolement.
    <br>
    <a href="https://solagro.org/images/imagesCK/files/documents/2021_Guide_me__thodologique_HVN.pdf"
        target="_blank" style="color: #1a73e8; text-decoration: underline;">
        Voir le guide méthodologique (PDF)
    </a>
</div>

<button id="desc-btn" onclick="
    var box = document.getElementById('desc-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer la description';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher la description';
    }
">Afficher la description</button>
"""
m.get_root().html.add_child(Element(html2))

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=2, right=2, width='150px').add_to(m)

m.save("carte_Timeslider_HVN_carreaux.html")